In [14]:
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
import time
import warnings

import pandas as pd

warnings.filterwarnings('ignore')

import os, sys
currentdir = os.path.dirname(os.path.abspath(''))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
from helper_functions import *

from helper_functions_constituent_alpha_for_dual_momo import  *

from datetime import timedelta
import os
import investpy
import yfinance as yf
import smtplib
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,10)
plt.rcParams['axes.grid'] = False
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# import eikon as ek
# ek.set_app_key('9a249e0411184cf49e553b61a6e76c52d295ec17')

def get_data_alpha(ticker):

    temp_og = get_data(ticker, "yfinance", "")

    today_data = yf.download(ticker, start=str(date.today() - timedelta(days=1)), interval="1m")

    if len(today_data)!=0:
        today_data.reset_index(inplace=True)
        today_data.drop(['Adj Close'], axis=1, inplace=True)

        today_time_close = today_data.iloc[-1]["Datetime"]

        temp_og = pd.concat([temp_og, pd.DataFrame([{"Date": pd.Timestamp(year=today_data.iloc[-1]["Datetime"].year,
                                                                          month=today_data.iloc[-1]["Datetime"].month,
                                                                          day=today_data.iloc[-1]["Datetime"].day),\
                                                     "Open": today_data.iloc[-1]["Open"],
                                                     "Close": today_data.iloc[-1]["Close"],
                                                     "High": today_data.iloc[-1]["High"],
                                                     "Low": today_data.iloc[-1]["Low"],
                                                     "Volume": today_data.iloc[-1]["Volume"]}])],axis=0).reset_index().drop(['index'], axis=1)
        temp_og.drop_duplicates(subset="Date",
                                keep='first', inplace=True)
    else:
        today_time_close = temp_og.iloc[-1]["Date"]

    temp_og = add_fisher(temp_og)
    return temp_og, today_time_close

def get_data_constituents(ticker, api,country):

    if api == "yfinance":

        temp_og = yf.download(ticker, start = '2007-07-01', end= str(date.today()+timedelta(1)))
        if len(temp_og)==0:
            temp_og = yf.download(ticker, start='2007-07-01', end=str(date.today()))
        temp_og.reset_index(inplace=True)
        temp_og.drop(['Adj Close'], axis=1, inplace=True)
        if ticker=="GOLDBEES.NS":
            temp_og = temp_og.loc[temp_og["Close"]>1]
        temp_og = add_fisher(temp_og)

    if api =="investpy":
        temp_og = get_data_investpy(symbol=ticker, country=country, from_date="01/07/2007",to_date=(date.today()+timedelta(1)).strftime("%d/%m/%Y"))
        temp_og.reset_index(inplace=True)
        temp_og = add_fisher(temp_og)

    if api == "reuters":
        temp_og = ek.get_timeseries(ticker, start_date='2007-07-01', end_date=str(date.today() + timedelta(1)))
        temp_og.reset_index(inplace=True)
        temp_og.rename(columns={"HIGH": "High", "CLOSE": "Close", "LOW": "Low", "OPEN": "Open", "VOLUME": "Volume"},
                       inplace=True)
        temp_og.drop(['COUNT'], axis=1, inplace=True)

    return temp_og

def get_data_alpha_investpy_yf(ticker, ticker_yfinance):

    temp_og = get_data_constituents(ticker, "investpy", "united states")

    today_data = yf.download(ticker_yfinance, start=str(date.today() - timedelta(days=1)), interval="1m")
    if len(today_data)==0:
        today_time_close = temp_og.iloc[-1]["Date"]
    else:
        today_data.reset_index(inplace=True)
        today_data.drop(['Adj Close'], axis=1, inplace=True)
        today_time_close = today_data.iloc[-1]["Datetime"]

        temp_og = pd.concat([temp_og, pd.DataFrame([{"Date": pd.Timestamp(year=today_data.iloc[-1]["Datetime"].year,
                                                                          month=today_data.iloc[-1]["Datetime"].month,
                                                                          day=today_data.iloc[-1]["Datetime"].day),\
                                                     "Open": today_data.iloc[-1]["Open"],
                                                     "Close": today_data.iloc[-1]["Close"],
                                                     "High": today_data.iloc[-1]["High"],
                                                     "Low": today_data.iloc[-1]["Low"],
                                                     "Volume": today_data.iloc[-1]["Volume"]}])],axis=0).reset_index().drop(['index'], axis=1)
        temp_og.drop_duplicates(subset="Date",
                            keep='first', inplace=True)
    temp_og = add_fisher(temp_og)
    return temp_og, today_time_close

def SendMail(subject_text_add, text_restricted,text_nifty, text_gold, text, printdf_nifty, printdf_gold, assets, ImgFileNameList):
    msg = MIMEMultipart()
    msg['Subject'] = f'{subject_text_add} [ADITYA INDIVIDUAL] NASDAQ - Strategy Update on Dual Momentum - Alpha & Accelerating Momentum'
    msg['From'] = 'algo_notifications@acsysindia.com'
    msg['Cc'] = 'suprabhashsahu@acsysindia.com, aditya@shankar.biz' #
    msg['To'] = 'algo_notifications@acsysindia.com'

    for ImgFileName in ImgFileNameList:
        with open(ImgFileName, 'rb') as f:
            img_data = f.read()
        image = MIMEImage(img_data, name=os.path.basename(ImgFileName))
        msg.attach(image)

    text_restricted = MIMEText(text_restricted)
    msg.attach(text_restricted)

    text_nifty = MIMEText(text_nifty)
    msg.attach(text_nifty)

    strategies_nifty = """\
    <html>
      <head></head>
      <body>
        {0}
      </body>
    </html>
    """.format(printdf_nifty.to_html())

    part1 = MIMEText(strategies_nifty, 'html')
    msg.attach(part1)

    text_gold = MIMEText(text_gold)
    msg.attach(text_gold)

    strategies_gold = """\
        <html>
          <head></head>
          <body>
            {0}
          </body>
        </html>
        """.format(printdf_gold.to_html())

    part2 = MIMEText(strategies_gold, 'html')
    msg.attach(part2)

    text = MIMEText(text)
    msg.attach(text)

    assetsprint = """\
            <html>
              <head></head>
              <body>
                {0}
              </body>
            </html>
            """.format(assets.to_html())

    part3 = MIMEText(assetsprint, 'html')
    msg.attach(part3)

    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.ehlo()
    s.starttls()
    s.ehlo()
    s.login('algo_notifications@acsysindia.com', 'esahYah8')
    s.sendmail('algo_notifications@acsysindia.com', ['suprabhashsahu@acsysindia.com', 'algo_notifications@acsysindia.com', 'aditya@shankar.biz'], msg.as_string())  #
    s.quit()

def signal_print(inp, ticker):
    if inp == 0:
        signal = f"Neutral on {ticker}, Long on Fixed Income"
    else:
        signal = f"Long on {ticker}, Neutral on Fixed Income"
    return signal

def execute_nasdaq(ticker, number_of_optimization_periods,recalib_months,num_strategies,metric):
    # Download data
    temp_og, today_time_close = get_data_alpha(ticker)

    dates_all_ss = pd.date_range(start=str(temp_og.iloc[1]['Date'] + timedelta(days=365))[:10], end="2024-06-15",
                                 freq=f'3M')
    dates_ss = valid_dates(dates_all_ss)

    print(f"Importing selected strategies: {datetime.now()}")
    with open(
            f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{number_of_optimization_periods}_Selected_Strategies_ss.pkl',
            'rb') as file:
        ss_test_imp = pickle.load(file)
    with open(
            f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{number_of_optimization_periods}_Selected_Strategies_res.pkl',
            'rb') as file:
        res_test_imp = pickle.load(file)

    res_test = []
    ss_test = []
    dates = []
    for date_i in range(len(dates_ss) - (int(24 / 3) + 1)):
        if (3 * date_i) % recalib_months == 0:
            dates.append(dates_ss[date_i + int(24 / 3)])
            ss_test.append(ss_test_imp[date_i])
            res_test.append(res_test_imp[date_i])

    dates.append(date.today()+timedelta(1))
    date_p = [date_i for date_i in range(len(dates) - 1)][-1]
    print(
        f"Selected Strategies for Testing period beginning: {str(dates[date_p])} and ending: {str(dates[date_p + 1])}")
    print(res_test[date_p])

    print(f"Importing Weights: {datetime.now()}")
    with open(
            f'{ticker}/weights/Results_Ticker{ticker}_LP{number_of_optimization_periods}_Recal{recalib_months}_NS{num_strategies}_M{metric}.pkl',
            'rb') as file:
        weights = pickle.load(file)

    inputs = []
    for date_i in range(len(dates) - 1):
        inputs.append(
            [date_i, dates, temp_og, ss_test, res_test, num_strategies, weights[date_i], recalib_months, dates_ss])
    try:
        pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
        results_backtest = pool.map(backtest_live, inputs)
    finally:  # To make sure processes are closed in the end, even if errors happen
        pool.close()
        pool.join()

    results_final = pd.DataFrame()
    for tt in results_backtest:
        results_final = pd.concat([results_final, tt[0]], axis=0)
    temp_res = results_final

    initial_amount = 2441200
    current_balance = initial_amount
    equity_allocation = 0
    cash_allocation = 0
    portfolio_value = pd.DataFrame()

    units_equity = 0

    for i in range(len(temp_res)):

        signal = temp_res["signal"].iloc[i]

        if signal == 1:
            equity_allocation = current_balance
            cash_allocation = 0
        else:
            equity_allocation = 0
            cash_allocation = current_balance

        if ((temp_res["signal"].iloc[i] == 1) & (temp_res["signal"].shift(1).fillna(0).iloc[i] == 0)):
            units_equity = equity_allocation / temp_res.iloc[i]["Close"]

        if ((temp_res["signal"].iloc[i] == 0) & (temp_res["signal"].shift(1).fillna(1).iloc[i] == 1)):
            units_equity = 0

        if signal == 1:
            equity_allocation = units_equity * temp_res.iloc[i]["Close"]

        cash_allocation = cash_allocation * (1 + 6 / 25200)
        current_balance = equity_allocation + cash_allocation
        portfolio_day = {'Date': temp_res.index[i], 'Signal_backtest': signal, 'units_equity': units_equity,
                         'equity_allocation': equity_allocation, 'cash_allocation': cash_allocation,
                         'Strategy_Return': current_balance}
        portfolio_day = pd.DataFrame([portfolio_day])
        portfolio_day = portfolio_day.set_index("Date")
        portfolio_value = pd.concat([portfolio_value, portfolio_day], axis=0, join="outer")

    temp_res = pd.concat([temp_res, portfolio_value], axis=1, join="inner")
    temp_res["Market_Return"] = (initial_amount / temp_res.iloc[0]["Close"]) * temp_res["Close"]
    temp_res.reset_index(inplace=True)
    temp_res_nasdaq = temp_res.copy()
    temp_res_nasdaq.columns = ["Date"] + [column + "_nasdaq" for column in temp_res_nasdaq.columns if column != 'Date']

    plt.plot(temp_res['Date'], temp_res['Market_Return'], color='black', label='Market Returns')
    plt.plot(temp_res['Date'], temp_res['Strategy_Return'], color='blue', label='Strategy Returns')
    plt.title('Strategy Backtest: Nasdaq Alpha')
    plt.legend(loc=0)
    plt.tight_layout()
    plt.savefig(f"Performance{ticker}.jpg")
    plt.clf()

    text = ""
    text = text + "*"*100 + "\n"
    text = text + "NASDAQ Alpha" + "\n"
    text = text + "Phase-1: Top strategies are ones which maximises the Average Win/Average Loss of the trades made over the lookback period" + "\n"
    text = text + "Top Strategies are selected based on lookbacks of 24 and 48 months" + "\n"
    text = text + f"Top Strategies are selected every {recalib_months} months" + "\n"
    text = text + f"Top {num_strategies} strategies are selected" + "\n"
    text = text + f"24-month Lookback Strategies selected based on training data from: {str(dates[date_p] - relativedelta(months=24))[:11]} to: {str(dates[date_p])[:11]} are selected" + "\n"
    text = text + f"48-month Lookback Strategies selected based on training data from: {str(dates[date_p] - relativedelta(months=48))[:11]} to: {str(dates[date_p])[:11]} are selected" + "\n"
    text = text + f"Phase-2: Weights of the selected strategies are calculated such that maxdrawup_by_maxdrawdown against benchmark is maximised over a lookback of 24 months "+ "\n"
    text = text + f"Last Recalibrated Nasdaq Alpha on {str(dates[-2])[:11]}" + "\n"
    text = text + f"Recalibrating Nasdaq Alpha on {str(dates[-2] + relativedelta(months = recalib_months))[:11]}" + "\n" + "\n"
    text = text + "Alpha Stats for last 252 trading days:" + "\n"
    text = text + f"Sortino: {np.round(backtest_sortino(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Sharpe: {np.round(backtest_sharpe(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Rolling Sortino: {np.round(backtest_rolling_sortino(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Rolling CAGR: {np.round(backtest_rolling_cagr(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"MaxDrawup/MaxDrawdown: {np.round(backtest_maxdrawup_by_maxdrawdown(temp_res[-252:], 0, 0), 2)}" + "\n" + "\n"
    text = text + f"Signal at : {str(today_time_close)[:19]}: {signal_print(temp_res.iloc[-1]['signal'],ticker)}" + "\n" f"Signal at Yesterday Close: {signal_print(temp_res.iloc[-2]['signal'],ticker)}" + "\n" f"Signal at Day before Yesterday Close: {signal_print(temp_res.iloc[-3]['signal'],ticker)}" + "\n" + "\n" + "Overall Performance:" + "\n" + f"Portfolio Value: {np.round(temp_res.iloc[-1]['Strategy_Return'], 2)}" + "\n" + "\n"
    text = text + "Selected Strategies: " + "\n" + "\n"
    return temp_og,str(today_time_close)[:19],dates_all_ss,dates_ss,results_backtest[-1][1],temp_res_nasdaq, text

def execute_tlt(ticker, number_of_optimization_periods,recalib_months,num_strategies,metric):

    # Download data
    temp_og, today_time_close = get_data_alpha(ticker)

    dates_all_ss = pd.date_range(start=str(temp_og.iloc[1]['Date'] + timedelta(days=365))[:10],
                                 end="2024-06-15", freq=f'3M')
    dates_ss = valid_dates(dates_all_ss)

    print(f"Importing selected strategies: {datetime.now()}")
    with open(
            f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{number_of_optimization_periods}_Selected_Strategies_ss.pkl',
            'rb') as file:
        ss_test_imp = pickle.load(file)
    with open(
            f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{number_of_optimization_periods}_Selected_Strategies_res.pkl',
            'rb') as file:
        res_test_imp = pickle.load(file)

    res_test = []
    ss_test = []
    dates = []
    for date_i in range(len(dates_ss) - (int(24 / 3) + 1)):
        if (3 * date_i) % recalib_months == 0:
            dates.append(dates_ss[date_i + int(24 / 3)])
            ss_test.append(ss_test_imp[date_i])
            res_test.append(res_test_imp[date_i])

    dates.append(date.today()+timedelta(1))
    date_p = [date_i for date_i in range(len(dates) - 1)][-1]
    print(
        f"Selected Strategies for Testing period beginning: {str(dates[date_p])} and ending: {str(dates[date_p + 1])}")
    print(res_test[date_p])

    print(f"Importing Weights: {datetime.now()}")
    with open(
            f'{ticker}/weights/Results_Ticker{ticker}_LP{number_of_optimization_periods}_Recal{recalib_months}_NS{num_strategies}_M{metric}.pkl',
            'rb') as file:
        weights = pickle.load(file)

    inputs = []
    for date_i in range(len(dates) - 1):
        inputs.append(
            [date_i, dates, temp_og, ss_test, res_test, num_strategies, weights[date_i], recalib_months,
             dates_ss])
    try:
        pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
        results_backtest = pool.map(backtest_live, inputs)
    finally:  # To make sure processes are closed in the end, even if errors happen
        pool.close()
        pool.join()

    results_final = pd.DataFrame()
    for tt in results_backtest:
        results_final = pd.concat([results_final, tt[0]], axis=0)
    temp_res = results_final

    initial_amount = 2441200
    current_balance = initial_amount
    equity_allocation = 0
    cash_allocation = 0
    portfolio_value = pd.DataFrame()

    units_equity = 0

    for i in range(len(temp_res)):

        signal = temp_res["signal"].iloc[i]

        if signal == 1:
            equity_allocation = current_balance
            cash_allocation = 0
        else:
            equity_allocation = 0
            cash_allocation = current_balance

        if ((temp_res["signal"].iloc[i] == 1) & (temp_res["signal"].shift(1).fillna(0).iloc[i] == 0)):
            units_equity = equity_allocation / temp_res.iloc[i]["Close"]

        if ((temp_res["signal"].iloc[i] == 0) & (temp_res["signal"].shift(1).fillna(1).iloc[i] == 1)):
            units_equity = 0

        if signal == 1:
            equity_allocation = units_equity * temp_res.iloc[i]["Close"]

        cash_allocation = cash_allocation * (1 + 6 / 25200)
        current_balance = equity_allocation + cash_allocation
        portfolio_day = {'Date': temp_res.index[i], 'Signal_backtest': signal, 'units_equity': units_equity,
                         'equity_allocation': equity_allocation, 'cash_allocation': cash_allocation,
                         'Strategy_Return': current_balance}
        portfolio_day = pd.DataFrame([portfolio_day])
        portfolio_day = portfolio_day.set_index("Date")
        portfolio_value = pd.concat([portfolio_value, portfolio_day], axis=0, join="outer")

    temp_res = pd.concat([temp_res, portfolio_value], axis=1, join="inner")
    temp_res["Market_Return"] = (initial_amount / temp_res.iloc[0]["Close"]) * temp_res["Close"]
    temp_res.reset_index(inplace=True)
    temp_res_tlt = temp_res.copy()
    temp_res_tlt.columns = ["Date"] + [column + "_tlt" for column in temp_res_tlt.columns if
                                        column != 'Date']

    plt.plot(temp_res['Date'], temp_res['Market_Return'], color='black', label='Market Returns')
    plt.plot(temp_res['Date'], temp_res['Strategy_Return'], color='blue', label='Strategy Returns')
    plt.title('Strategy Backtest: TLT Alpha')
    plt.legend(loc=0)
    plt.tight_layout()
    plt.savefig(f"Performance{ticker}.jpg")
    plt.clf()

    text = ""
    text = text + "*" * 100 + "\n"
    text = text + "TLT Alpha" + "\n"
    text = text + "Phase-1: Top strategies are ones which maximises the Average Win/Average Loss of the trades made over the lookback period" + "\n"
    text = text + "Top Strategies are selected based on lookbacks of 24 and 48 months" + "\n"
    text = text + f"Top Strategies are selected every {recalib_months} months" + "\n"
    text = text + f"Top {num_strategies} strategies are selected" + "\n"
    text = text + f"24-month Lookback Strategies selected based on training data from: {str(dates[date_p] - relativedelta(months=24))[:11]} to: {str(dates[date_p])[:11]} are selected" + "\n"
    text = text + f"48-month Lookback Strategies selected based on training data from: {str(dates[date_p] - relativedelta(months=48))[:11]} to: {str(dates[date_p])[:11]} are selected" + "\n"
    text = text + f"Phase-2: Weights of the selected strategies are calculated such that rolling sharpe against benchmark is maximised over a lookback of 24 months " + "\n"
    text = text + f"Last Recalibrated TLT Alpha on {str(dates[-2])[:11]}" + "\n"
    text = text + f"Recalibrating TLT Alpha on {str(dates[-2] + relativedelta(months = recalib_months))[:11]}" + "\n" + "\n"
    text = text + "Alpha Stats for last 252 trading days:" + "\n"
    text = text + f"Sortino: {np.round(backtest_sortino(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Sharpe: {np.round(backtest_sharpe(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Rolling Sortino: {np.round(backtest_rolling_sortino(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Rolling CAGR: {np.round(backtest_rolling_cagr(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"MaxDrawup/MaxDrawdown: {np.round(backtest_maxdrawup_by_maxdrawdown(temp_res[-252:], 0, 0), 2)}" + "\n" + "\n"
    text = text + f"Signal at : {str(today_time_close)[:19]} : {signal_print(temp_res.iloc[-1]['signal'],ticker)}" + "\n" f"Signal at Yesterday Close: {signal_print(temp_res.iloc[-2]['signal'],ticker)}" + "\n" f"Signal at Day before Yesterday Close: {signal_print(temp_res.iloc[-3]['signal'],ticker)}" + "\n" + "\n" + "Overall Performance:" + "\n" + f"Portfolio Value: {np.round(temp_res.iloc[-1]['Strategy_Return'], 2)}" + "\n" + "\n"
    text = text + "Selected Strategies: " + "\n" + "\n"
    return temp_og,str(today_time_close)[:19],dates_all_ss,dates_ss,results_backtest[-1][1],temp_res_tlt, text

def execute_constituents(ticker, ticker_yfinance, number_of_optimization_periods,recalib_months,num_strategies,metric):

    # Download data
    temp_og, today_time_close = get_data_alpha_investpy_yf(ticker, ticker_yfinance)

    dates_all_ss = pd.date_range(start=str(temp_og.iloc[1]['Date'] + timedelta(days=365))[:10],
                                 end="2024-06-15", freq=f'3M')
    dates_ss = valid_dates(dates_all_ss)

    print(f"Importing selected strategies: {datetime.now()}")
    with open(
            f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{number_of_optimization_periods}_Selected_Strategies_ss.pkl',
            'rb') as file:
        ss_test_imp = pickle.load(file)
    with open(
            f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{number_of_optimization_periods}_Selected_Strategies_res.pkl',
            'rb') as file:
        res_test_imp = pickle.load(file)

    res_test = []
    ss_test = []
    dates = []
    for date_i in range(len(dates_ss) - (int(24 / 3) + 1)):
        if (3 * date_i) % recalib_months == 0:
            dates.append(dates_ss[date_i + int(24 / 3)])
            ss_test.append(ss_test_imp[date_i])
            res_test.append(res_test_imp[date_i])

    dates.append(date.today()+timedelta(1))
    date_p = [date_i for date_i in range(len(dates) - 1)][-1]
    print(
        f"Selected Strategies for Testing period beginning: {str(dates[date_p])} and ending: {str(dates[date_p + 1])}")
    print(res_test[date_p])

    print(f"Importing Weights: {datetime.now()}")
    with open(
            f'{ticker}/weights/Results_Ticker{ticker}_LP{number_of_optimization_periods}_Recal{recalib_months}_NS{num_strategies}_M{metric}.pkl',
            'rb') as file:
        weights = pickle.load(file)

    inputs = []
    for date_i in range(len(dates) - 1):
        inputs.append(
            [date_i, dates, temp_og, ss_test, res_test, num_strategies, weights[date_i], recalib_months,
             dates_ss])
    try:
        pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
        results_backtest = pool.map(backtest_live, inputs)
    finally:  # To make sure processes are closed in the end, even if errors happen
        pool.close()
        pool.join()

    results_final = pd.DataFrame()
    for tt in results_backtest:
        results_final = pd.concat([results_final, tt[0]], axis=0)
    temp_res = results_final

    temp_res['Return'] = np.log(temp_res['Close'] / temp_res['Close'].shift(1))
    temp_res['Market_Return'] = temp_res['Return'].expanding().sum()
    temp_res['Strategy_Return'] = temp_res['S_Return'].expanding().sum()
    temp_res['Portfolio Value'] = ((temp_res['Strategy_Return'] + 1) * 10000)
    temp_res.reset_index(inplace=True)
    temp_res_ticker = temp_res.copy()
    temp_res_ticker.columns = ["Date"] + [column + ticker for column in temp_res_ticker.columns if
                                        column != 'Date']

    plt.plot(temp_res['Date'], temp_res['Market_Return'], color='black', label='Market Returns')
    plt.plot(temp_res['Date'], temp_res['Strategy_Return'], color='blue', label='Strategy Returns')
    plt.title(f'Strategy Backtest: {ticker} Alpha')
    plt.legend(loc=0)
    plt.tight_layout()
    plt.savefig(f"Performance{ticker}.jpg")
    plt.clf()

    return temp_og,str(today_time_close)[:19],dates_all_ss,dates_ss,results_backtest[-1][1],temp_res_ticker

In [20]:
ticker_nasdaq = "^IXIC"
number_of_optimization_periods_nasdaq = 2
recalib_months_nasdaq = 3
num_strategies_nasdaq = 5
metric_nasdaq = 'maxdrawup_by_maxdrawdown'

ticker_tlt = "TLT"
number_of_optimization_periods_tlt = 2
recalib_months_tlt = 6
num_strategies_tlt = 5
metric_tlt = 'rolling_sharpe'

run_hour = 20
run_minute = 57

text = ""
print(f"Executing: {datetime.now()}")

text_restricted = "Restricted List is : [PGHH, PFIZER]" + "\n"

index = ".NDX"
with open(f'NDX_Constituents.pkl', 'rb') as file:
    constituents = pickle.load(file)
constituents_all = []
for i in range(len(constituents)):
    constituents_all = constituents_all + constituents.iloc[i][0]
tickers = list(set(constituents_all))
tickers = [ticker.partition('.')[0] for ticker in tickers]
# Restricted Stock
#tickers.remove('AXBK')

# removing duplicate tickers
tickers_remove = ['DISCK', 'FOXA', 'GOOG', 'LBTYK', 'LILAK', 'TFCFA']
for remove in tickers_remove:
    tickers.remove(remove)

#remove tickers for which data isn't available on investing.com
tickers_remove = ['MAR', 'SHPG', 'ESRX', 'MRNA', 'FOX', 'PTON', 'VIAB', 'ZM', 'WLTW', 'TFCF', 'CRWD', 'RHT', 'AABA', 'CELG', 'CA', 'EFII', 'LVNTA', 'ALXN', 'MXIM']
for remove in tickers_remove:
    tickers.remove(remove)

tickers = tickers+ ["SBAC"]

recalibrating_months = 1
training_period = 24  # 24/48/96
dates_recalibrating = valid_dates(
    pd.date_range(start="2007-01-01", end="2024-06-15", freq=f'{recalibrating_months}M'))
data_inp = prepare_portfolio_data(tickers, recalibrating_months, "investpy", "united states")

with open(f'NASDAQ_RecalibPeriod_{int(1)}.pkl', 'rb') as file:
    assets = pickle.load(file)

temp_og_nasdaq, signal_time_nasdaq, dates_all_ss_nasdaq, dates_ss_nasdaq, printdf_nasdaq, temp_res_nasdaq, text_nasdaq = execute_nasdaq(
    ticker_nasdaq, number_of_optimization_periods_nasdaq, recalib_months_nasdaq, num_strategies_nasdaq,
    metric_nasdaq)
temp_og_tlt, signal_time_tlt, dates_all_ss_tlt, dates_ss_tlt, printdf_tlt, temp_res_tlt, text_tlt = execute_tlt(
    ticker_tlt, number_of_optimization_periods_tlt, recalib_months_tlt, num_strategies_tlt, metric_tlt)


Executing: 2021-11-12 21:18:45.360941
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Importing selected strategies: 2021-11-12 21:28:14.546196
Selected Strategies for Testing period beginning: 2021-10-31 00:00:00 and ending: 2021-11-13
   Lookback  Low Bound  High Bound  AvgWinLoss  Optimization_Years
0       290       -0.8        -1.2     4473.00                 4.0
1       390        1.8         0.2     3244.50                 4.0
2        70       -1.0        -1.0      295.07                 2.0
3        90        3.5         0.2      273.18                 2.0
4        50       -1.0         0.5      270.29                 2.0
5        70       -3.5        -1.5      191.66                 4.0
6       130        0.8         1.5      144.71                 2.0
7       370        1.8         2.5      144.57                 2.0
8       290        6.5         6.5       93.50                 2.0
9  

<Figure size 1080x360 with 0 Axes>

In [21]:
if temp_res_nasdaq.iloc[-1]['signal_nasdaq'] == 1:
    subject_text_add = "[Long on Index]"
if (temp_res_nasdaq.iloc[-1]['signal_nasdaq'] == 0) & (temp_res_tlt.iloc[-1]['signal_tlt'] == 1):
    subject_text_add = "[Long on Gold]"
if (temp_res_nasdaq.iloc[-1]['signal_nasdaq'] == 0) & (temp_res_tlt.iloc[-1]['signal_tlt'] == 0):
    subject_text_add = "[Long on Cash]"

data_inp_backtest = pd.concat([data_inp.set_index('Date'), temp_res_nasdaq.set_index('Date'), temp_res_tlt.set_index('Date')], axis=1, join='inner').reset_index()
data_inp_backtest.loc[(data_inp_backtest['Date'] == '2016-07-21'), 'DXCM'] = 84.63

In [22]:
constituent_alpha_params = {
    'ROST': {'ticker_yfinance': 'ROST',
  'number_of_optimization_periods': 1,
  'recalib_months': 6,
  'num_strategies': 7,
  'metric': 'rolling_sharpe'},
 'MNST': {'ticker_yfinance': 'MNST',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 5,
  'metric': 'rolling_sharpe'},
 'CMCSA': {'ticker_yfinance': 'CMCSA',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'rolling_cagr'},
 'KLAC': {'ticker_yfinance': 'KLAC',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 5,
  'metric': 'rolling_sortino'},
 'NXPI': {'ticker_yfinance': 'NXPI',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 5,
  'metric': 'rolling_sortino'},
 'SHPG': {'ticker_yfinance': 'SHPG',
  'number_of_optimization_periods': 0,
  'recalib_months': 0,
  'num_strategies': 0,
  'metric': ''},
 'XLNX': {'ticker_yfinance': 'XLNX',
  'number_of_optimization_periods': 1,
  'recalib_months': 6,
  'num_strategies': 3,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'ALGN': {'ticker_yfinance': 'ALGN',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 5,
  'metric': 'rolling_cagr'},
 'MRVL': {'ticker_yfinance': 'MRVL',
  'number_of_optimization_periods': 1,
  'recalib_months': 6,
  'num_strategies': 5,
  'metric': 'rolling_sharpe'},
 'ISRG': {'ticker_yfinance': 'ISRG',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 1,
  'metric': 'rolling_sharpe'},
 'MAT': {'ticker_yfinance': 'MAT',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 7,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'OKTA': {'ticker_yfinance': 'OKTA',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 3,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'AVGO': {'ticker_yfinance': 'AVGO',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 3,
  'metric': 'rolling_sharpe'},
 'DXCM': {'ticker_yfinance': 'DXCM',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 5,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'AMD': {'ticker_yfinance': 'AMD',
  'number_of_optimization_periods': 3,
  'recalib_months': 6,
  'num_strategies': 3,
  'metric': 'rolling_sortino'},
 'DOCU': {'ticker_yfinance': 'DOCU',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 7,
  'metric': 'rolling_cagr'},
 'INTC': {'ticker_yfinance': 'INTC',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 3,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'UAL': {'ticker_yfinance': 'UAL',
  'number_of_optimization_periods': 1,
  'recalib_months': 6,
  'num_strategies': 7,
  'metric': 'rolling_cagr'},
 'KDP': {'ticker_yfinance': 'KDP',
  'number_of_optimization_periods': 3,
  'recalib_months': 6,
  'num_strategies': 5,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'WBA': {'ticker_yfinance': 'WBA',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 7,
  'metric': 'rolling_cagr'},
 'CSCO': {'ticker_yfinance': 'CSCO',
  'number_of_optimization_periods': 2,
  'recalib_months': 6,
  'num_strategies': 7,
  'metric': 'rolling_sortino'},
 'SIRI': {'ticker_yfinance': 'SIRI',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 1,
  'metric': 'outperformance'},
 'LRCX': {'ticker_yfinance': 'LRCX',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 3,
  'metric': 'rolling_sharpe'},
 'GILD': {'ticker_yfinance': 'GILD',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'outperformance'},
 'ADP': {'ticker_yfinance': 'ADP',
  'number_of_optimization_periods': 3,
  'recalib_months': 6,
  'num_strategies': 5,
  'metric': 'outperformance'},
 'NLOK': {'ticker_yfinance': 'NLOK',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 5,
  'metric': 'rolling_sharpe'},
 'ADSK': {'ticker_yfinance': 'ADSK',
  'number_of_optimization_periods': 2,
  'recalib_months': 12,
  'num_strategies': 1,
  'metric': 'rolling_sortino'},
 'AMZN': {'ticker_yfinance': 'AMZN',
  'number_of_optimization_periods': 2,
  'recalib_months': 12,
  'num_strategies': 3,
  'metric': 'outperformance'},
 'QRTEA': {'ticker_yfinance': 'QRTEA',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'outperformance'},
 'REGN': {'ticker_yfinance': 'REGN',
  'number_of_optimization_periods': 3,
  'recalib_months': 6,
  'num_strategies': 7,
  'metric': 'outperformance'},
 'LBTYA': {'ticker_yfinance': 'LBTYA',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 5,
  'metric': 'outperformance'},
 'TMUS': {'ticker_yfinance': 'TMUS',
  'number_of_optimization_periods': 3,
  'recalib_months': 6,
  'num_strategies': 3,
  'metric': 'rolling_sortino'},
 'LULU': {'ticker_yfinance': 'LULU',
  'number_of_optimization_periods': 2,
  'recalib_months': 3,
  'num_strategies': 5,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'SGEN': {'ticker_yfinance': 'SGEN',
  'number_of_optimization_periods': 2,
  'recalib_months': 6,
  'num_strategies': 7,
  'metric': 'rolling_sharpe'},
 'MDLZ': {'ticker_yfinance': 'MDLZ',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 1,
  'metric': 'rolling_sortino'},
 'INCY': {'ticker_yfinance': 'INCY',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 3,
  'metric': 'rolling_sortino'},
 'TCOM': {'ticker_yfinance': 'TCOM',
  'number_of_optimization_periods': 1,
  'recalib_months': 6,
  'num_strategies': 5,
  'metric': 'rolling_sharpe'},
 'STX': {'ticker_yfinance': 'STX',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'rolling_sharpe'},
 'CDNS': {'ticker_yfinance': 'CDNS',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 1,
  'metric': 'rolling_sharpe'},
 'NTAP': {'ticker_yfinance': 'NTAP',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 7,
  'metric': 'outperformance'},
 'HAS': {'ticker_yfinance': 'HAS',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 5,
  'metric': 'rolling_sharpe'},
 'CHTR': {'ticker_yfinance': 'CHTR',
  'number_of_optimization_periods': 1,
  'recalib_months': 6,
  'num_strategies': 5,
  'metric': 'outperformance'},
 'ILMN': {'ticker_yfinance': 'ILMN',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 3,
  'metric': 'rolling_sortino'},
 'SBUX': {'ticker_yfinance': 'SBUX',
  'number_of_optimization_periods': 2,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'rolling_sortino'},
 'PYPL': {'ticker_yfinance': 'PYPL',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'rolling_sharpe'},
 'EBAY': {'ticker_yfinance': 'EBAY',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 3,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'AMGN': {'ticker_yfinance': 'AMGN',
  'number_of_optimization_periods': 2,
  'recalib_months': 3,
  'num_strategies': 5,
  'metric': 'rolling_cagr'},
 'TEAM': {'ticker_yfinance': 'TEAM',
  'number_of_optimization_periods': 2,
  'recalib_months': 3,
  'num_strategies': 3,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'MCHP': {'ticker_yfinance': 'MCHP',
  'number_of_optimization_periods': 1,
  'recalib_months': 6,
  'num_strategies': 5,
  'metric': 'rolling_sortino'},
 'BIDU': {'ticker_yfinance': 'BIDU',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 7,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'NCLH': {'ticker_yfinance': 'NCLH',
  'number_of_optimization_periods': 2,
  'recalib_months': 3,
  'num_strategies': 3,
  'metric': 'rolling_sharpe'},
 'EA': {'ticker_yfinance': 'EA',
  'number_of_optimization_periods': 3,
  'recalib_months': 6,
  'num_strategies': 1,
  'metric': 'rolling_sortino'},
 'XEL': {'ticker_yfinance': 'XEL',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 7,
  'metric': 'rolling_sortino'},
 'CERN': {'ticker_yfinance': 'CERN',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 5,
  'metric': 'rolling_cagr'},
 'CDW': {'ticker_yfinance': 'CDW',
  'number_of_optimization_periods': 2,
  'recalib_months': 6,
  'num_strategies': 1,
  'metric': 'rolling_sortino'},
 'AMAT': {'ticker_yfinance': 'AMAT',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 5,
  'metric': 'outperformance'},
 'CPRT': {'ticker_yfinance': 'CPRT',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'outperformance'},
 'BKNG': {'ticker_yfinance': 'BKNG',
  'number_of_optimization_periods': 2,
  'recalib_months': 6,
  'num_strategies': 5,
  'metric': 'rolling_sharpe'},
 'CTSH': {'ticker_yfinance': 'CTSH',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 1,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'AEP': {'ticker_yfinance': 'AEP',
  'number_of_optimization_periods': 2,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'CHKP': {'ticker_yfinance': 'CHKP',
  'number_of_optimization_periods': 2,
  'recalib_months': 3,
  'num_strategies': 5,
  'metric': 'rolling_sortino'},
 'PEP': {'ticker_yfinance': 'PEP',
  'number_of_optimization_periods': 3,
  'recalib_months': 6,
  'num_strategies': 5,
  'metric': 'rolling_sortino'},
 'FB': {'ticker_yfinance': 'FB',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 3,
  'metric': 'rolling_cagr'},
 'JD': {'ticker_yfinance': 'JD',
  'number_of_optimization_periods': 2,
  'recalib_months': 6,
  'num_strategies': 7,
  'metric': 'rolling_sortino'},
 'ANSS': {'ticker_yfinance': 'ANSS',
  'number_of_optimization_periods': 2,
  'recalib_months': 6,
  'num_strategies': 7,
  'metric': 'rolling_sharpe'},
 'VTRS': {'ticker_yfinance': 'VTRS',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 3,
  'metric': 'rolling_sortino'},
 'INTU': {'ticker_yfinance': 'INTU',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 3,
  'metric': 'outperformance'},
 'LILA': {'ticker_yfinance': 'LILA',
  'number_of_optimization_periods': 3,
  'recalib_months': 6,
  'num_strategies': 7,
  'metric': 'rolling_cagr'},
 'CSGP': {'ticker_yfinance': 'CSGP',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 7,
  'metric': 'rolling_sharpe'},
 'NVDA': {'ticker_yfinance': 'NVDA',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 5,
  'metric': 'rolling_sharpe'},
 'GOOGL': {'ticker_yfinance': 'GOOGL',
  'number_of_optimization_periods': 3,
  'recalib_months': 6,
  'num_strategies': 5,
  'metric': 'outperformance'},
 'VOD': {'ticker_yfinance': 'VOD',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'NFLX': {'ticker_yfinance': 'NFLX',
  'number_of_optimization_periods': 2,
  'recalib_months': 12,
  'num_strategies': 5,
  'metric': 'rolling_sharpe'},
 'JBHT': {'ticker_yfinance': 'JBHT',
  'number_of_optimization_periods': 1,
  'recalib_months': 6,
  'num_strategies': 1,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'XRAY': {'ticker_yfinance': 'XRAY',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 3,
  'metric': 'rolling_sharpe'},
 'DLTR': {'ticker_yfinance': 'DLTR',
  'number_of_optimization_periods': 1,
  'recalib_months': 6,
  'num_strategies': 7,
  'metric': 'outperformance'},
 'VRTX': {'ticker_yfinance': 'VRTX',
  'number_of_optimization_periods': 1,
  'recalib_months': 6,
  'num_strategies': 5,
  'metric': 'rolling_cagr'},
 'COST': {'ticker_yfinance': 'COST',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 3,
  'metric': 'outperformance'},
 'IDXX': {'ticker_yfinance': 'IDXX',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 3,
  'metric': 'rolling_cagr'},
 'TTWO': {'ticker_yfinance': 'TTWO',
  'number_of_optimization_periods': 2,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'rolling_sortino'},
 'FISV': {'ticker_yfinance': 'FISV',
  'number_of_optimization_periods': 2,
  'recalib_months': 3,
  'num_strategies': 1,
  'metric': 'rolling_sortino'},
 'AKAM': {'ticker_yfinance': 'AKAM',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'rolling_sharpe'},
 'ADBE': {'ticker_yfinance': 'ADBE',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 5,
  'metric': 'rolling_sharpe'},
 'NTES': {'ticker_yfinance': 'NTES',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 5,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'BIIB': {'ticker_yfinance': 'BIIB',
  'number_of_optimization_periods': 2,
  'recalib_months': 6,
  'num_strategies': 5,
  'metric': 'rolling_cagr'},
 'SWKS': {'ticker_yfinance': 'SWKS',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 1,
  'metric': 'rolling_sharpe'},
 'SNPS': {'ticker_yfinance': 'SNPS',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'rolling_cagr'},
 'AAL': {'ticker_yfinance': 'AAL',
  'number_of_optimization_periods': 1,
  'recalib_months': 6,
  'num_strategies': 5,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'EXC': {'ticker_yfinance': 'EXC',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 1,
  'metric': 'rolling_cagr'},
 'DISH': {'ticker_yfinance': 'DISH',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 5,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'MU': {'ticker_yfinance': 'MU',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 3,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'VRSN': {'ticker_yfinance': 'VRSN',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 1,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'QCOM': {'ticker_yfinance': 'QCOM',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 3,
  'metric': 'outperformance'},
 'TSCO': {'ticker_yfinance': 'TSCO',
  'number_of_optimization_periods': 2,
  'recalib_months': 6,
  'num_strategies': 7,
  'metric': 'outperformance'},
 'MELI': {'ticker_yfinance': 'MELI',
  'number_of_optimization_periods': 2,
  'recalib_months': 3,
  'num_strategies': 5,
  'metric': 'rolling_sortino'},
 'HOLX': {'ticker_yfinance': 'HOLX',
  'number_of_optimization_periods': 2,
  'recalib_months': 12,
  'num_strategies': 3,
  'metric': 'rolling_sortino'},
 'WYNN': {'ticker_yfinance': 'WYNN',
  'number_of_optimization_periods': 1,
  'recalib_months': 6,
  'num_strategies': 3,
  'metric': 'rolling_sortino'},
 'EXPE': {'ticker_yfinance': 'EXPE',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 5,
  'metric': 'rolling_cagr'},
 'BMRN': {'ticker_yfinance': 'BMRN',
  'number_of_optimization_periods': 2,
  'recalib_months': 3,
  'num_strategies': 3,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'FAST': {'ticker_yfinance': 'FAST',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 5,
  'metric': 'outperformance'},
 'ASML': {'ticker_yfinance': 'ASML',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 1,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'TSLA': {'ticker_yfinance': 'TSLA',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'KHC': {'ticker_yfinance': 'KHC',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 3,
  'metric': 'rolling_cagr'},
 'MSFT': {'ticker_yfinance': 'MSFT',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 5,
  'metric': 'rolling_cagr'},
 'ORLY': {'ticker_yfinance': 'ORLY',
  'number_of_optimization_periods': 1,
  'recalib_months': 3,
  'num_strategies': 5,
  'metric': 'outperformance'},
 'PAYX': {'ticker_yfinance': 'PAYX',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'rolling_sortino'},
 'CTXS': {'ticker_yfinance': 'CTXS',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 5,
  'metric': 'rolling_cagr'},
 'PCAR': {'ticker_yfinance': 'PCAR',
  'number_of_optimization_periods': 2,
  'recalib_months': 3,
  'num_strategies': 7,
  'metric': 'outperformance'},
 'ULTA': {'ticker_yfinance': 'ULTA',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'CSX': {'ticker_yfinance': 'CSX',
  'number_of_optimization_periods': 1,
  'recalib_months': 6,
  'num_strategies': 5,
  'metric': 'rolling_sharpe'},
 'DISCA': {'ticker_yfinance': 'DISCA',
  'number_of_optimization_periods': 2,
  'recalib_months': 12,
  'num_strategies': 1,
  'metric': 'rolling_sharpe'},
 'WDC': {'ticker_yfinance': 'WDC',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 7,
  'metric': 'rolling_cagr'},
 'ADI': {'ticker_yfinance': 'ADI',
  'number_of_optimization_periods': 3,
  'recalib_months': 6,
  'num_strategies': 1,
  'metric': 'rolling_sharpe'},
 'HSIC': {'ticker_yfinance': 'HSIC',
  'number_of_optimization_periods': 2,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'TXN': {'ticker_yfinance': 'TXN',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 7,
  'metric': 'rolling_sharpe'},
 'ATVI': {'ticker_yfinance': 'ATVI',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'rolling_cagr'},
 'AAPL': {'ticker_yfinance': 'AAPL',
  'number_of_optimization_periods': 2,
  'recalib_months': 12,
  'num_strategies': 3,
  'metric': 'rolling_cagr'},
 'CTAS': {'ticker_yfinance': 'CTAS',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 3,
  'metric': 'outperformance'},
 'SPLK': {'ticker_yfinance': 'SPLK',
  'number_of_optimization_periods': 2,
  'recalib_months': 6,
  'num_strategies': 5,
  'metric': 'rolling_cagr'},
 'HON': {'ticker_yfinance': 'HON',
  'number_of_optimization_periods': 3,
  'recalib_months': 3,
  'num_strategies': 7,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'VRSK': {'ticker_yfinance': 'VRSK',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 5,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'PDD': {'ticker_yfinance': 'PDD',
  'number_of_optimization_periods': 2,
  'recalib_months': 12,
  'num_strategies': 5,
  'metric': 'rolling_sharpe'},
 'TRIP': {'ticker_yfinance': 'TRIP',
  'number_of_optimization_periods': 3,
  'recalib_months': 12,
  'num_strategies': 7,
  'metric': 'outperformance'},
 'MTCH': {'ticker_yfinance': 'MTCH',
  'number_of_optimization_periods': 2,
  'recalib_months': 12,
  'num_strategies': 5,
  'metric': 'maxdrawup_by_maxdrawdown'},
 'WDAY': {'ticker_yfinance': 'WDAY',
  'number_of_optimization_periods': 1,
  'recalib_months': 6,
  'num_strategies': 7,
  'metric': 'rolling_sharpe'},
  'SBAC': {'ticker_yfinance': 'SBAC',
  'number_of_optimization_periods': 1,
  'recalib_months': 12,
  'num_strategies': 5,
  'metric': 'outperformance'}}

In [23]:
for i, ticker in enumerate(tickers):
    temp_og_constituent, _, _, _, _, temp_res_constituent = execute_constituents(ticker, constituent_alpha_params[ticker]["ticker_yfinance"],constituent_alpha_params[ticker]["number_of_optimization_periods"],constituent_alpha_params[ticker]["recalib_months"],constituent_alpha_params[ticker]["num_strategies"],constituent_alpha_params[ticker]["metric"])
    if i==0:
        constituent_signals = temp_res_constituent[["Date", f"signal{ticker}"]].set_index("Date")
    else:
        constituent_signals = pd.concat([constituent_signals, temp_res_constituent[["Date", f"signal{ticker}"]].set_index("Date")], axis=1, join="outer")

data_inp_backtest = pd.concat([data_inp_backtest.set_index("Date"), constituent_signals], axis=1, join="inner")
data_inp_backtest.reset_index(inplace=True)


[*********************100%***********************]  1 of 1 completed
Importing selected strategies: 2021-11-12 21:28:55.295562
Selected Strategies for Testing period beginning: 2021-07-31 00:00:00 and ending: 2021-11-13
   Lookback  Low Bound  High Bound  AvgWinLoss  Optimization_Years
0        70        3.0         0.2     1945.00                 2.0
1       110       -1.5        -0.5      464.75                 2.0
2       230        4.5         5.2      190.00                 2.0
3       250        4.5         4.5      170.67                 2.0
4       150       -1.5        -1.0      157.88                 2.0
5        50        3.0        -0.8      127.20                 2.0
6       390        1.5         2.2      120.69                 2.0
7        50       -3.0         1.0      102.33                 2.0
8       210        3.8         1.5       92.80                 2.0
9        90        2.5        -0.2       82.93                 2.0
Importing Weights: 2021-11-12 21:28:55.3055

<Figure size 1080x360 with 0 Axes>

In [84]:
for top_nassets in [8,9,10]:
    for rebalancing_months in [3,6,12,18]:
        for starting_point in range(rebalancing_months):

            if (top_nassets,rebalancing_months,starting_point) not in [(8,12,5),(8,12,6),(8,18,5),(8,18,6),(8,18,8),(8,18,12),(9,12,6),(9,18,5),(9,18,6),(9,18,12),(10,18,8),(10,18,12)]:
                continue
            dates_rebalancing = []
            assetsb = []
            for date_i in range(len(dates_recalibrating) - (int(training_period / recalibrating_months) + 1)):
                if ((recalibrating_months * date_i) % rebalancing_months == starting_point) & (
                        dates_recalibrating[date_i + int(training_period / recalibrating_months)] >=
                        data_inp_backtest["Date"][0]):
                    dates_rebalancing.append(
                        dates_recalibrating[date_i + int(training_period / recalibrating_months)])
                    assetsb.append(assets[date_i].iloc[:top_nassets])
            dates_rebalancing.append(date.today() + timedelta(1))

            results_final, unit_ticker, unit_tlt = backtest_Alpha_AM_NASDAQ(dates_rebalancing, data_inp_backtest,
                                                                            assetsb)

            bench = yf.download("^IXIC", start='2007-01-01', end=str(date.today() + timedelta(1)))
            bench = bench.loc[bench["Close"] > 1]
            bench["Return"] = np.log(bench["Close"] / bench["Close"].shift(1))

            results_final["S_Return"] = pd.DataFrame(
                np.log(results_final["Pvalue"] / results_final["Pvalue"].shift(1)))
            temp_res = pd.concat([results_final, bench["Return"]], join="inner", axis=1)
            temp_res['Market_Return'] = np.exp(temp_res['Return'].expanding().sum()) - 1
            temp_res['Strategy_Return'] = np.exp(temp_res['S_Return'].expanding().sum()) - 1
            temp_res['Portfolio Value'] = temp_res['Pvalue']
            temp_res = temp_res.reset_index().rename(columns={'index': "Date"})

            portfolio = results_final["Pvalue"]
            routperformance = rolling_outperformance(portfolio, naive_portfolio_value)

            fig, (ax1, ax2) = plt.subplots(2)
            ax1.plot(temp_res['Date'], temp_res['Market_Return'], color='blacxk', label='Market Returns')
            ax1.plot(temp_res['Date'], temp_res['Strategy_Return'], color='blue', label='Strategy Returns')
            fig.suptitle('Strategy Backtest: Dual Momentum')
            ax1.legend(loc=0)
            ax2.plot(routperformance, label="rolling 1 year outperformance")
            ax2.legend(loc=0)
            plt.savefig(f"NASDAQ_Tests/Topnassets{top_nassets}RM{rebalancing_months}SP{starting_point}.jpg")
            plt.clf()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

...

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

<Figure size 1080x720 with 0 Axes>

In [ ]:
                text = text + "*" * 100 + "\n"
                text = text + "DUAL MOMENTUM STRATEGY" + "\n"
                text = text + f"Recalibrated every {rebalancing_months} months, with training data of {training_period - recalibrating_months} months with a forward return  of {recalibrating_months} month" + "\n"
                text = text + f"The number of assets being selected are: {top_nassets}"
                text = text + f"Last Recalibrated Accelerating Momentum on {str(dates_rebalancing[-2])[:11]}" + "\n"
                text = text + f"Recalibrating Accelerating Momentum on {str(dates_rebalancing[-2] + relativedelta(months=rebalancing_months))[:11]}" + "\n" + "\n"
                text = text + f"Strategies selected based on training data from: {str(dates_rebalancing[-2] - relativedelta(months=training_period))[:11]} to: {str(dates_rebalancing[-2] - relativedelta(months=recalibrating_months))[:11]} are selected" + "\n"
                text = text + f"Units and Momentum Values were calculated on {str(dates_rebalancing[-2])[:11]}" + "\n"
                text = text + "On Rebalancing Day, if NASDAQ Alpha is long, 100% of the Portolfio are allocated to the stocks below.\nIf NASDAQ Alpha is Neutral and Gold Alpha is Long, 50% of the Portfolio is allocated to TLT and 50% to Fixed Income.\nIf both Alphas are Neutral, 100% of the Portfolio are allocated to Fixed Income" + "\n" + "\n"
                text = text + "Stats for last 252 trading days:" + "\n"
                text = text + f"Sortino: {np.round(backtest_sortino(temp_res[-252:], 0, 0), 2)}" + "\n"
                text = text + f"Sharpe: {np.round(backtest_sharpe(temp_res[-252:], 0, 0), 2)}" + "\n"
                text = text + f"Rolling Sortino: {np.round(backtest_rolling_sortino(temp_res[-252:], 0, 0), 2)}" + "\n"
                text = text + f"Rolling Sharpe: {np.round(backtest_rolling_sharpe(temp_res[-252:], 0, 0), 2)}" + "\n"
                text = text + f"Rolling CAGR: {np.round(backtest_rolling_cagr(temp_res[-252:], 0, 0), 2)}" + "\n"
                text = text + f"Rolling MaxDrawup/MaxDrawdown: {np.round(backtest_maxdrawup_by_maxdrawdown(temp_res[-252:], 0, 0), 2)}" + "\n"
                text = text + f"Rolling Outperformance: {np.round(backtest_outperformance(temp_res[-252:], 0, 0), 2)}"
                text = text + "\n" + "\n" + "Overall Performance:" + "\n"
                text = text + f" A Portfolio of Rs {np.round(temp_res.iloc[0]['Portfolio Value'] / 10000000, 4)} Cr invested on: {str(dates_rebalancing[0])[:11]} is now valued at: {np.round(temp_res.iloc[-1]['Portfolio Value'] / 10000000, 4)} Cr." + "\n" + "\n"
                text = text + "Selected Assets: " + "\n"
                text = text + f"Number of units of GOLDBEES.NS: {unit_gold}" + "\n"

                emailer_consts = pd.concat([assetsb[-1][["Ticker", "Accelerating Momentum"]].set_index("Ticker"),
                                            pd.DataFrame([unit_ticker]).transpose().rename(columns={0: "Units"})],
                                           axis=1)

                emailer_consts["Signal"] = 1

                for ticker in list(assetsb[-1]["Ticker"]):
                    if constituent_signals[f"signal{ticker}"].iloc[-1] == 0:
                        emailer_consts.loc[ticker, "Units"] = 0
                        emailer_consts.loc[ticker, "Signal"] = 0

                const_alpha_plots = []
                for ticker in list(assetsb[-1]["Ticker"]):
                    const_alpha_plots.append(f"Performance{ticker}.jpg")

                const_alpha_plots = const_alpha_plots + ["Performance^IXIC.jpg", "PerformanceTLT.jpg","PerformanceAMNASDAQConst.jpg"]

                # if running_for_the_first_time == True:
                #     pass
                # else:
                SendMail(subject_text_add, text_restricted, text_nasdaq, text_tlt, text, printdf_nasdaq, printdf_tlt, emailer_consts, const_alpha_plots)

            if running_for_the_first_time:
                pass
            else:
                if datetime.now() < datetime.now().replace(hour=run_hour).replace(minute=run_minute).replace(second=30):
                    continue

            running_for_the_first_time = False

            print(f"Sleeping: {datetime.now()}")

            time_now = datetime.now()
            next_run = datetime.now()
            try:
                if (datetime.now().hour < run_hour) & (datetime.now().minute < run_minute):
                    next_run = next_run.replace(day=next_run.day).replace(hour=run_hour).replace(
                        minute=run_minute).replace(
                        second=00)
                else:
                    next_run = next_run.replace(day=next_run.day + 1).replace(hour=run_hour).replace(
                        minute=run_minute).replace(second=00)
            except:
                if next_run.month == 12:
                    next_run = next_run.replace(day=1).replace(month=1).replace(year=next_run.year + 1).replace(
                        hour=run_hour).replace(minute=run_minute).replace(second=00)
                else:
                    next_run = next_run.replace(day=1).replace(month=next_run.month + 1).replace(hour=run_hour).replace(
                        minute=run_minute).replace(second=00)

            print(f"Supposed to wake up at: {datetime.now() + timedelta(seconds=(next_run - time_now).seconds - 150)}")
            time.sleep((next_run - time_now).seconds - 150)
            print(f"Woken Up: {datetime.now()}")




In [15]:
import time
import warnings
warnings.filterwarnings('ignore')

import os, sys
currentdir = os.path.dirname(os.path.abspath(''))
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)
from helper_functions import *

from datetime import timedelta
import os
import investpy
import yfinance as yf
import smtplib
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from email.mime.multipart import MIMEMultipart
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['axes.grid'] = False
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

# import eikon as ek
# ek.set_app_key('9a249e0411184cf49e553b61a6e76c52d295ec17')

def get_data_alpha(ticker):

    temp_og = get_data(ticker, "yfinance", "")

    today_data = yf.download(ticker, start=str(date.today() - timedelta(days=1)), interval="1m")

    if len(today_data)!=0:
        today_data.reset_index(inplace=True)
        today_data.drop(['Adj Close'], axis=1, inplace=True)

        today_time_close = today_data.iloc[-1]["Datetime"]

        temp_og = pd.concat([temp_og, pd.DataFrame([{"Date": pd.Timestamp(year=today_data.iloc[-1]["Datetime"].year,
                                                                          month=today_data.iloc[-1]["Datetime"].month,
                                                                          day=today_data.iloc[-1]["Datetime"].day),\
                                                     "Open": today_data.iloc[-1]["Open"],
                                                     "Close": today_data.iloc[-1]["Close"],
                                                     "High": today_data.iloc[-1]["High"],
                                                     "Low": today_data.iloc[-1]["Low"],
                                                     "Volume": today_data.iloc[-1]["Volume"]}])],axis=0).reset_index().drop(['index'], axis=1)
        temp_og.drop_duplicates(subset="Date",
                                keep='first', inplace=True)
    else:
        today_time_close = temp_og.iloc[-1]["Date"]

    temp_og = add_fisher(temp_og)
    return temp_og, today_time_close

def select_all_strategies(train_monthsf, datesf, temp_ogf, ticker, save=True):
    inputs =[]
    for date_i in range(len(datesf)-(int(train_monthsf/3)+1)):
        inputs.append([date_i, datesf, temp_ogf, train_monthsf])
    try:
        pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
        results = pool.map(get_strategies_brute_force, inputs)
    finally: # To make sure processes are closed in the end, even if errors happen
        pool.close()
        pool.join()

    res_test_update = [pd.DataFrame(columns=["Lookback", "Low Bound", "High Bound", "AvgWinLoss",\
                                       "Optimization_Years"])] * (len(datesf)-(int(24/3)+1))

    for i in range(len(results)):
        res_test_update[results[i][0]+int((train_monthsf-24)/3)] = pd.concat([res_test_update[results[i][0]],results[i][1].reset_index().drop(['index'], axis=1)], axis=0)

    if save==True:
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_{int(train_monthsf/12)}_All_Strategies.pkl','rb') as file:
            res_test = pickle.load(file)
        res_test.append(res_test_update[-1])
        with open(f'{ticker}/SelectedStrategies/{ticker}_TrainYrs_{int(train_monthsf/12)}_All_Strategies.pkl', 'wb') as file:
            pickle.dump(res_test, file)
    return res_test

def select_strategies_from_corr_filter(res_testf2,res_testf4,res_testf8, datesf, temp_ogf, num_opt_periodsf,num_strategiesf, ticker, save=True):
    train_monthsf = 24  #minimum optimization lookback
    res_total = [None]*(len(datesf)-(int(train_monthsf/3)+1))
    for i in range(len(datesf)-(int(train_monthsf/3)+1)):
        if num_opt_periodsf==1:
            res_total[i] = pd.concat([res_testf2[i]], axis = 0)
        if num_opt_periodsf==2:
            res_total[i] = pd.concat([res_testf2[i],res_testf4[i]], axis=0)
        if num_opt_periodsf==3:
            res_total[i] = pd.concat([res_testf2[i],res_testf4[i],res_testf8[i]], axis=0)
        res_total[i] = res_total[i].reset_index().drop(['index'], axis=1)

    ss_test_update = [None]*(len(datesf)-(int(train_monthsf/3)+1))
    res_test_update = [None]*(len(datesf)-(int(train_monthsf/3)+1))
    inputs = []
    for date_i in range(len(datesf)-(int(train_monthsf/3)+1)):
        inputs.append([date_i, datesf, temp_ogf,res_total, num_strategiesf,train_monthsf])
    try:
        pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
        results_filtered = pool.map(corr_sortino_filter, inputs)
    finally: # To make sure processes are closed in the end, even if errors happen
        pool.close()
        pool.join()

    for i in range(len(datesf)-(int(train_monthsf/3)+1)):
        ss_test_update[results_filtered[i][0]] = results_filtered[i][1]
        res_test_update[results_filtered[i][0]] = results_filtered[i][2]

    if save==True:
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{num_opt_periodsf}_Selected_Strategies_ss.pkl','rb') as file:
            ss_test = pickle.load(file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{num_opt_periodsf}_Selected_Strategies_res.pkl','rb') as file:
            res_test = pickle.load(file)

        ss_test.append(ss_test_update[-1])
        res_test.append(res_test_update[-1])

        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{num_opt_periodsf}_Selected_Strategies_ss.pkl', 'wb') as file:
            pickle.dump(ss_test, file)
        with open(f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{num_opt_periodsf}_Selected_Strategies_res.pkl', 'wb') as file:
            pickle.dump(res_test, file)

    return ss_test, res_test

def SendMail(subject_text_add, text_restricted,text_nasdaq, text_tlt, text, printdf_nasdaq, printdf_tlt, assets, ImgFileNameList):
    msg = MIMEMultipart()
    msg['Subject'] = f'{subject_text_add} [ADITYA INDIVIDUAL] Strategy Update on NASDAQ Dual Momentum - Alpha & Accelerating Momentum'
    msg['From'] = 'algo_notifications@acsysindia.com'
    msg['Cc'] = 'suprabhashsahu@acsysindia.com, aditya@shankar.biz' #
    msg['To'] = 'algo_notifications@acsysindia.com'

    for ImgFileName in ImgFileNameList:
        with open(ImgFileName, 'rb') as f:
            img_data = f.read()
        image = MIMEImage(img_data, name=os.path.basename(ImgFileName))
        msg.attach(image)

    text_restricted = MIMEText(text_restricted)
    msg.attach(text_restricted)

    text_nasdaq = MIMEText(text_nasdaq)
    msg.attach(text_nasdaq)

    strategies_nasdaq = """\
    <html>
      <head></head>
      <body>
        {0}
      </body>
    </html>
    """.format(printdf_nasdaq.to_html())

    part1 = MIMEText(strategies_nasdaq, 'html')
    msg.attach(part1)

    text_tlt = MIMEText(text_tlt)
    msg.attach(text_tlt)

    strategies_tlt = """\
        <html>
          <head></head>
          <body>
            {0}
          </body>
        </html>
        """.format(printdf_tlt.to_html())

    part2 = MIMEText(strategies_tlt, 'html')
    msg.attach(part2)

    text = MIMEText(text)
    msg.attach(text)

    assetsprint = """\
            <html>
              <head></head>
              <body>
                {0}
              </body>
            </html>
            """.format(assets.to_html())

    part3 = MIMEText(assetsprint, 'html')
    msg.attach(part3)

    s = smtplib.SMTP('smtp.gmail.com', 587)
    s.ehlo()
    s.starttls()
    s.ehlo()
    s.login('algo_notifications@acsysindia.com', 'esahYah8')
    s.sendmail('algo_notifications@acsysindia.com', ['suprabhashsahu@acsysindia.com', 'algo_notifications@acsysindia.com', 'aditya@shankar.biz'], msg.as_string())  #
    s.quit()

def signal_print(inp, ticker):
    if inp == 0:
        signal = f"Neutral on {ticker}, Long on Fixed Income"
    else:
        signal = f"Long on {ticker}, Neutral on Fixed Income"
    return signal

def execute_nasdaq(ticker, number_of_optimization_periods,recalib_months,num_strategies,metric):
    # Download data
    temp_og, today_time_close = get_data_alpha(ticker)

    dates_all_ss = pd.date_range(start=str(temp_og.iloc[1]['Date'] + timedelta(days=365))[:10], end="2024-06-15",
                                 freq=f'3M')
    dates_ss = valid_dates(dates_all_ss)

    print(f"Importing selected strategies: {datetime.now()}")
    with open(
            f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{number_of_optimization_periods}_Selected_Strategies_ss.pkl',
            'rb') as file:
        ss_test_imp = pickle.load(file)
    with open(
            f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{number_of_optimization_periods}_Selected_Strategies_res.pkl',
            'rb') as file:
        res_test_imp = pickle.load(file)

    res_test = []
    ss_test = []
    dates = []
    for date_i in range(len(dates_ss) - (int(24 / 3) + 1)):
        if (3 * date_i) % recalib_months == 0:
            dates.append(dates_ss[date_i + int(24 / 3)])
            ss_test.append(ss_test_imp[date_i])
            res_test.append(res_test_imp[date_i])

    dates.append(date.today()+timedelta(1))
    date_p = [date_i for date_i in range(len(dates) - 1)][-1]
    print(
        f"Selected Strategies for Testing period beginning: {str(dates[date_p])} and ending: {str(dates[date_p + 1])}")
    print(res_test[date_p])

    print(f"Importing Weights: {datetime.now()}")
    with open(
            f'{ticker}/weights/Results_Ticker{ticker}_LP{number_of_optimization_periods}_Recal{recalib_months}_NS{num_strategies}_M{metric}.pkl',
            'rb') as file:
        weights = pickle.load(file)

    inputs = []
    for date_i in range(len(dates) - 1):
        inputs.append(
            [date_i, dates, temp_og, ss_test, res_test, num_strategies, weights[date_i], recalib_months, dates_ss])
    try:
        pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
        results_backtest = pool.map(backtest_live, inputs)
    finally:  # To make sure processes are closed in the end, even if errors happen
        pool.close()
        pool.join()

    results_final = pd.DataFrame()
    for tt in results_backtest:
        results_final = pd.concat([results_final, tt[0]], axis=0)
    temp_res = results_final

    initial_amount = 2441200
    current_balance = initial_amount
    equity_allocation = 0
    cash_allocation = 0
    portfolio_value = pd.DataFrame()

    units_equity = 0

    for i in range(len(temp_res)):

        signal = temp_res["signal"].iloc[i]

        if signal == 1:
            equity_allocation = current_balance
            cash_allocation = 0
        else:
            equity_allocation = 0
            cash_allocation = current_balance

        if ((temp_res["signal"].iloc[i] == 1) & (temp_res["signal"].shift(1).fillna(0).iloc[i] == 0)):
            units_equity = equity_allocation / temp_res.iloc[i]["Close"]

        if ((temp_res["signal"].iloc[i] == 0) & (temp_res["signal"].shift(1).fillna(1).iloc[i] == 1)):
            units_equity = 0

        if signal == 1:
            equity_allocation = units_equity * temp_res.iloc[i]["Close"]

        cash_allocation = cash_allocation * (1 + 6 / 25200)
        current_balance = equity_allocation + cash_allocation
        portfolio_day = {'Date': temp_res.index[i], 'Signal_backtest': signal, 'units_equity': units_equity,
                         'equity_allocation': equity_allocation, 'cash_allocation': cash_allocation,
                         'Strategy_Return': current_balance}
        portfolio_day = pd.DataFrame([portfolio_day])
        portfolio_day = portfolio_day.set_index("Date")
        portfolio_value = pd.concat([portfolio_value, portfolio_day], axis=0, join="outer")

    temp_res = pd.concat([temp_res, portfolio_value], axis=1, join="inner")
    temp_res["Market_Return"] = (initial_amount / temp_res.iloc[0]["Close"]) * temp_res["Close"]
    temp_res.reset_index(inplace=True)
    temp_res_nasdaq = temp_res.copy()
    temp_res_nasdaq.columns = ["Date"] + [column + "_nasdaq" for column in temp_res_nasdaq.columns if column != 'Date']

    plt.plot(temp_res['Date'], temp_res['Market_Return'], color='black', label='Market Returns')
    plt.plot(temp_res['Date'], temp_res['Strategy_Return'], color='blue', label='Strategy Returns')
    plt.title('Strategy Backtest: Nasdaq Alpha')
    plt.legend(loc=0)
    plt.tight_layout()
    plt.savefig(f"Performance{ticker}.jpg")
    plt.clf()

    text = ""
    text = text + "*"*100 + "\n"
    text = text + "NASDAQ Alpha" + "\n"
    text = text + "Phase-1: Top strategies are ones which maximises the Average Win/Average Loss of the trades made over the lookback period" + "\n"
    text = text + "Top Strategies are selected based on lookbacks of 24 and 48 months" + "\n"
    text = text + f"Top Strategies are selected every {recalib_months} months" + "\n"
    text = text + f"Top {num_strategies} strategies are selected" + "\n"
    text = text + f"24-month Lookback Strategies selected based on training data from: {str(dates[date_p] - relativedelta(months=24))[:11]} to: {str(dates[date_p])[:11]} are selected" + "\n"
    text = text + f"48-month Lookback Strategies selected based on training data from: {str(dates[date_p] - relativedelta(months=48))[:11]} to: {str(dates[date_p])[:11]} are selected" + "\n"
    text = text + f"Phase-2: Weights of the selected strategies are calculated such that maxdrawup_by_maxdrawdown against benchmark is maximised over a lookback of 24 months "+ "\n"
    text = text + f"Last Recalibrated Nasdaq Alpha on {str(dates[-2])[:11]}" + "\n"
    text = text + f"Recalibrating Nasdaq Alpha on {str(dates[-2] + relativedelta(months = recalib_months))[:11]}" + "\n" + "\n"
    text = text + "Alpha Stats for last 252 trading days:" + "\n"
    text = text + f"Sortino: {np.round(backtest_sortino(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Sharpe: {np.round(backtest_sharpe(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Rolling Sortino: {np.round(backtest_rolling_sortino(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Rolling CAGR: {np.round(backtest_rolling_cagr(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"MaxDrawup/MaxDrawdown: {np.round(backtest_maxdrawup_by_maxdrawdown(temp_res[-252:], 0, 0), 2)}" + "\n" + "\n"
    text = text + f"Signal at : {str(today_time_close)[:19]}: {signal_print(temp_res.iloc[-1]['signal'],ticker)}" + "\n" f"Signal at Yesterday Close: {signal_print(temp_res.iloc[-2]['signal'],ticker)}" + "\n" f"Signal at Day before Yesterday Close: {signal_print(temp_res.iloc[-3]['signal'],ticker)}" + "\n" + "\n" + "Overall Performance:" + "\n" + f"Portfolio Value: {np.round(temp_res.iloc[-1]['Strategy_Return'], 2)}" + "\n" + "\n"
    text = text + "Selected Strategies: " + "\n" + "\n"
    return temp_og,str(today_time_close)[:19],dates_all_ss,dates_ss,results_backtest[-1][1],temp_res_nasdaq, text

def execute_tlt(ticker, number_of_optimization_periods,recalib_months,num_strategies,metric):

    # Download data
    temp_og, today_time_close = get_data_alpha(ticker)

    dates_all_ss = pd.date_range(start=str(temp_og.iloc[1]['Date'] + timedelta(days=365))[:10],
                                 end="2024-06-15", freq=f'3M')
    dates_ss = valid_dates(dates_all_ss)

    print(f"Importing selected strategies: {datetime.now()}")
    with open(
            f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{number_of_optimization_periods}_Selected_Strategies_ss.pkl',
            'rb') as file:
        ss_test_imp = pickle.load(file)
    with open(
            f'{ticker}/SelectedStrategies/{ticker}_OptPeriods_{number_of_optimization_periods}_Selected_Strategies_res.pkl',
            'rb') as file:
        res_test_imp = pickle.load(file)

    res_test = []
    ss_test = []
    dates = []
    for date_i in range(len(dates_ss) - (int(24 / 3) + 1)):
        if (3 * date_i) % recalib_months == 0:
            dates.append(dates_ss[date_i + int(24 / 3)])
            ss_test.append(ss_test_imp[date_i])
            res_test.append(res_test_imp[date_i])

    dates.append(date.today()+timedelta(1))
    date_p = [date_i for date_i in range(len(dates) - 1)][-1]
    print(
        f"Selected Strategies for Testing period beginning: {str(dates[date_p])} and ending: {str(dates[date_p + 1])}")
    print(res_test[date_p])

    print(f"Importing Weights: {datetime.now()}")
    with open(
            f'{ticker}/weights/Results_Ticker{ticker}_LP{number_of_optimization_periods}_Recal{recalib_months}_NS{num_strategies}_M{metric}.pkl',
            'rb') as file:
        weights = pickle.load(file)

    inputs = []
    for date_i in range(len(dates) - 1):
        inputs.append(
            [date_i, dates, temp_og, ss_test, res_test, num_strategies, weights[date_i], recalib_months,
             dates_ss])
    try:
        pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
        results_backtest = pool.map(backtest_live, inputs)
    finally:  # To make sure processes are closed in the end, even if errors happen
        pool.close()
        pool.join()

    results_final = pd.DataFrame()
    for tt in results_backtest:
        results_final = pd.concat([results_final, tt[0]], axis=0)
    temp_res = results_final

    initial_amount = 2441200
    current_balance = initial_amount
    equity_allocation = 0
    cash_allocation = 0
    portfolio_value = pd.DataFrame()

    units_equity = 0

    for i in range(len(temp_res)):

        signal = temp_res["signal"].iloc[i]

        if signal == 1:
            equity_allocation = current_balance
            cash_allocation = 0
        else:
            equity_allocation = 0
            cash_allocation = current_balance

        if ((temp_res["signal"].iloc[i] == 1) & (temp_res["signal"].shift(1).fillna(0).iloc[i] == 0)):
            units_equity = equity_allocation / temp_res.iloc[i]["Close"]

        if ((temp_res["signal"].iloc[i] == 0) & (temp_res["signal"].shift(1).fillna(1).iloc[i] == 1)):
            units_equity = 0

        if signal == 1:
            equity_allocation = units_equity * temp_res.iloc[i]["Close"]

        cash_allocation = cash_allocation * (1 + 6 / 25200)
        current_balance = equity_allocation + cash_allocation
        portfolio_day = {'Date': temp_res.index[i], 'Signal_backtest': signal, 'units_equity': units_equity,
                         'equity_allocation': equity_allocation, 'cash_allocation': cash_allocation,
                         'Strategy_Return': current_balance}
        portfolio_day = pd.DataFrame([portfolio_day])
        portfolio_day = portfolio_day.set_index("Date")
        portfolio_value = pd.concat([portfolio_value, portfolio_day], axis=0, join="outer")

    temp_res = pd.concat([temp_res, portfolio_value], axis=1, join="inner")
    temp_res["Market_Return"] = (initial_amount / temp_res.iloc[0]["Close"]) * temp_res["Close"]
    temp_res.reset_index(inplace=True)
    temp_res_tlt = temp_res.copy()
    temp_res_tlt.columns = ["Date"] + [column + "_tlt" for column in temp_res_tlt.columns if
                                        column != 'Date']

    plt.plot(temp_res['Date'], temp_res['Market_Return'], color='black', label='Market Returns')
    plt.plot(temp_res['Date'], temp_res['Strategy_Return'], color='blue', label='Strategy Returns')
    plt.title('Strategy Backtest: TLT Alpha')
    plt.legend(loc=0)
    plt.tight_layout()
    plt.savefig(f"Performance{ticker}.jpg")
    plt.clf()

    text = ""
    text = text + "*" * 100 + "\n"
    text = text + "TLT Alpha" + "\n"
    text = text + "Phase-1: Top strategies are ones which maximises the Average Win/Average Loss of the trades made over the lookback period" + "\n"
    text = text + "Top Strategies are selected based on lookbacks of 24 and 48 months" + "\n"
    text = text + f"Top Strategies are selected every {recalib_months} months" + "\n"
    text = text + f"Top {num_strategies} strategies are selected" + "\n"
    text = text + f"24-month Lookback Strategies selected based on training data from: {str(dates[date_p] - relativedelta(months=24))[:11]} to: {str(dates[date_p])[:11]} are selected" + "\n"
    text = text + f"48-month Lookback Strategies selected based on training data from: {str(dates[date_p] - relativedelta(months=48))[:11]} to: {str(dates[date_p])[:11]} are selected" + "\n"
    text = text + f"Phase-2: Weights of the selected strategies are calculated such that rolling sharpe against benchmark is maximised over a lookback of 24 months " + "\n"
    text = text + f"Last Recalibrated TLT Alpha on {str(dates[-2])[:11]}" + "\n"
    text = text + f"Recalibrating TLT Alpha on {str(dates[-2] + relativedelta(months = recalib_months))[:11]}" + "\n" + "\n"
    text = text + "Alpha Stats for last 252 trading days:" + "\n"
    text = text + f"Sortino: {np.round(backtest_sortino(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Sharpe: {np.round(backtest_sharpe(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Rolling Sortino: {np.round(backtest_rolling_sortino(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"Rolling CAGR: {np.round(backtest_rolling_cagr(temp_res[-252:], 0, 0), 2)}" + "\n"
    text = text + f"MaxDrawup/MaxDrawdown: {np.round(backtest_maxdrawup_by_maxdrawdown(temp_res[-252:], 0, 0), 2)}" + "\n" + "\n"
    text = text + f"Signal at : {str(today_time_close)[:19]} : {signal_print(temp_res.iloc[-1]['signal'],ticker)}" + "\n" f"Signal at Yesterday Close: {signal_print(temp_res.iloc[-2]['signal'],ticker)}" + "\n" f"Signal at Day before Yesterday Close: {signal_print(temp_res.iloc[-3]['signal'],ticker)}" + "\n" + "\n" + "Overall Performance:" + "\n" + f"Portfolio Value: {np.round(temp_res.iloc[-1]['Strategy_Return'], 2)}" + "\n" + "\n"
    text = text + "Selected Strategies: " + "\n" + "\n"
    return temp_og,str(today_time_close)[:19],dates_all_ss,dates_ss,results_backtest[-1][1],temp_res_tlt, text

In [16]:
ticker_nasdaq = "^IXIC"
number_of_optimization_periods_nasdaq = 2
recalib_months_nasdaq = 3
num_strategies_nasdaq = 5
metric_nasdaq = 'maxdrawup_by_maxdrawdown'

ticker_tlt = "TLT"
number_of_optimization_periods_tlt = 2
recalib_months_tlt = 6
num_strategies_tlt = 5
metric_tlt = 'rolling_sharpe'

run_hour = 20
run_minute = 57

text = ""
print(f"Executing: {datetime.now()}")

text_restricted = "Restricted List is : [PGHH, PFIZER]" + "\n"

index = ".NDX"
with open(f'NDX_Constituents.pkl', 'rb') as file:
    constituents = pickle.load(file)
constituents_all = []
for i in range(len(constituents)):
    constituents_all = constituents_all + constituents.iloc[i][0]
tickers = list(set(constituents_all))
tickers = [ticker.partition('.')[0] for ticker in tickers]
# Restricted Stock
#tickers.remove('AXBK')

# removing duplicate tickers
tickers_remove = ['DISCK', 'FOXA', 'GOOG', 'LBTYK', 'LILAK', 'TFCFA']
for remove in tickers_remove:
    tickers.remove(remove)

recalibrating_months = 1
top_nassets = 8
training_period = 24  # 24/48/96
dates_recalibrating = valid_dates(pd.date_range(start="2007-01-01", end="2024-06-15", freq=f'{recalibrating_months}M'))
data_inp = prepare_portfolio_data(tickers, recalibrating_months, "investpy", "united states")

with open(f'NASDAQ_RecalibPeriod_{int(1)}.pkl', 'rb') as file:
    assets = pickle.load(file)

temp_og_nasdaq, signal_time_nasdaq, dates_all_ss_nasdaq, dates_ss_nasdaq, printdf_nasdaq, temp_res_nasdaq, text_nasdaq = execute_nasdaq(ticker_nasdaq,number_of_optimization_periods_nasdaq,recalib_months_nasdaq,num_strategies_nasdaq,metric_nasdaq)
temp_og_tlt, signal_time_tlt, dates_all_ss_tlt, dates_ss_tlt, printdf_tlt, temp_res_tlt, text_tlt = execute_tlt(ticker_tlt,number_of_optimization_periods_tlt,recalib_months_tlt,num_strategies_tlt,metric_tlt)

if temp_res_nasdaq.iloc[-1]['signal_nasdaq'] == 1:
    subject_text_add = "[Long on Index]"
if (temp_res_nasdaq.iloc[-1]['signal_nasdaq'] == 0) & (temp_res_tlt.iloc[-1]['signal_tlt'] == 1):
    subject_text_add = "[Long on Gold]"
if (temp_res_nasdaq.iloc[-1]['signal_nasdaq'] == 0) & (temp_res_tlt.iloc[-1]['signal_tlt'] == 0):
    subject_text_add = "[Long on Cash]"

data_inp_backtest = pd.concat([data_inp.set_index('Date'), temp_res_nasdaq.set_index('Date'), temp_res_tlt.set_index('Date')], axis=1, join='inner').reset_index()

rebalancing_months = 12
dates_rebalancing = []
assetsb = []
for date_i in range(len(dates_recalibrating) - (int(training_period / recalibrating_months) + 1)):
    if ((recalibrating_months * date_i) % rebalancing_months == 2) & (
            dates_recalibrating[date_i + int(training_period / recalibrating_months)] >=
            data_inp_backtest["Date"][0]):
        dates_rebalancing.append(dates_recalibrating[date_i + int(training_period / recalibrating_months)])
        assetsb.append(assets[date_i].iloc[:top_nassets])
dates_rebalancing.append(date.today()+timedelta(1))

data_inp_backtest.loc[(data_inp_backtest['Date'] == '2016-07-21'), 'DXCM'] = 84.63

results_final, unit_ticker, unit_tlt = backtest_Alpha_AM_NASDAQ(dates_rebalancing, data_inp_backtest, assetsb)

Executing: 2021-11-12 19:03:29.883473
LVNTA not processed
MXIM not processed
AABA not processed
RHT not processed
TFCF not processed
CELG not processed
ESRX not processed
SHPG not processed
EFII not processed
ALXN not processed
CA not processed
VIAB not processed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Importing selected strategies: 2021-11-12 19:19:17.014259
Selected Strategies for Testing period beginning: 2021-10-31 00:00:00 and ending: 2021-11-13
   Lookback  Low Bound  High Bound  AvgWinLoss  Optimization_Years
0       290       -0.8        -1.2     4473.00                 4.0
1       390        1.8         0.2     3244.50                 4.0
2        70       -1.0        -1.0      295.07                 2.0
3        90        3.5         0.2      273.18                 2.0
4        50       -1.0         0.5      270.29                 2.0
5        70       -3.5        -1.5      191.6

<Figure size 1080x360 with 0 Axes>

In [17]:
results_final["Pvalue"]

Date
2010-03-31     39364.000000
2010-04-01     39477.178867
2010-04-05     39955.323350
2010-04-06     40127.588433
2010-04-07     39607.850862
                  ...      
2021-11-05    425847.585335
2021-11-08    431737.115881
2021-11-09    431301.590021
2021-11-10    419618.784228
2021-11-11    421345.357350
Name: Pvalue, Length: 2927, dtype: float64

In [18]:
naive_portfolio_value = results_final["Pvalue"]

In [ ]:
                bench = yf.download("^IXIC", start='2007-01-01', end=str(date.today()+timedelta(1)))
                bench = bench.loc[bench["Close"] > 1]
                bench["Return"] = np.log(bench["Close"] / bench["Close"].shift(1))

                results_final["S_Return"] = pd.DataFrame(np.log(results_final["Pvalue"] / results_final["Pvalue"].shift(1)))
                temp_res = pd.concat([results_final, bench["Return"]], join="inner", axis=1)
                temp_res['Market_Return'] = np.exp(temp_res['Return'].expanding().sum())-1
                temp_res['Strategy_Return'] = np.exp(temp_res['S_Return'].expanding().sum())-1
                temp_res['Portfolio Value'] = temp_res['Pvalue']
                temp_res = temp_res.reset_index().rename(columns={'index': "Date"})

                plt.plot(temp_res['Date'], temp_res['Market_Return'], color='black', label='Market Returns')
                plt.plot(temp_res['Date'], temp_res['Strategy_Return'], color='blue', label='Strategy Returns')
                plt.title('Strategy Backtest: Dual Momentum')
                plt.legend(loc=0)
                plt.tight_layout()
                plt.savefig("PerformanceNASDAQ.jpg")
                plt.clf()

                text = text + "*" * 100+ "\n"
                text = text + "DUAL MOMENTUM STRATEGY"+ "\n"
                text = text + f"Recalibrated every {rebalancing_months} months, with training data of {training_period-recalibrating_months} months with a forward return  of {recalibrating_months} month"+ "\n"
                text = text + f"The number of assets being selected are: {top_nassets}" + "\n"
                text = text + f"Last Recalibrated Accelerating Momentum on {str(dates_rebalancing[-2])[:11]}" + "\n"
                text = text + f"Recalibrating Accelerating Momentum on {str(dates_rebalancing[-2] + relativedelta(months = rebalancing_months))[:11]}" + "\n" + "\n"
                text = text + f"Strategies selected based on training data from: {str(dates_rebalancing[-2]- relativedelta(months = training_period))[:11]} to: {str(dates_rebalancing[-2] - relativedelta(months = recalibrating_months))[:11]} are selected" + "\n"
                text = text + f"Units and Momentum Values were calculated on {str(dates_rebalancing[-2])[:11]}" + "\n"
                text = text + "On Rebalancing Day, if Nasdaq Alpha is long, 100% of the Portolfio are allocated to the stocks below.\nIf Nasdaq Alpha is Neutral and TLT Alpha is Long, 50% of the Portfolio is allocated to TLT and 50% to Fixed Income.\nIf both Alphas are Neutral, 100% of the Portfolio are allocated to Fixed Income" + "\n"+ "\n"
                text = text + "Stats for last 252 trading days:" + "\n"
                text = text + f"Sortino: {np.round(backtest_sortino(temp_res[-252:], 0, 0), 2)}"+ "\n"
                text = text + f"Sharpe: {np.round(backtest_sharpe(temp_res[-252:], 0, 0), 2)}"+ "\n"
                text = text + f"Rolling Sortino: {np.round(backtest_rolling_sortino(temp_res[-252:], 0, 0), 2)}"+ "\n"
                text = text + f"Rolling Sharpe: {np.round(backtest_rolling_sharpe(temp_res[-252:], 0, 0), 2)}"+ "\n"
                text = text + f"Rolling CAGR: {np.round(backtest_rolling_cagr(temp_res[-252:], 0, 0), 2)}"+ "\n"
                text = text + f"Rolling MaxDrawup/MaxDrawdown: {np.round(backtest_maxdrawup_by_maxdrawdown(temp_res[-252:], 0, 0), 2)}"+ "\n"
                text = text + f"Rolling Outperformance: {np.round(backtest_outperformance(temp_res[-252:], 0, 0), 2)}"
                text = text + "\n" + "\n" + "Overall Performance:" + "\n"
                text = text + f" A Portfolio of USD {np.round(temp_res.iloc[0]['Portfolio Value']/1000, 4)}K invested on: {str(dates_rebalancing[0])[:11]} is now valued at: USD {np.round(temp_res.iloc[-1]['Portfolio Value']/1000, 4)}K " + "\n" + "\n"
                text = text + "Selected Assets: " + "\n"
                text = text + f"Number of units of TLT: {unit_tlt}"+ "\n"
                # text = f"Signal at : {str(today_time_close)[:19]} : {signal_print(temp_res.iloc[-1]['signal'])}" + "\n" f"Signal at Yesterday Close: {signal_print(temp_res.iloc[-2]['signal'])}" + "\n" f"Signal at Day before Yesterday Close: {signal_print(temp_res.iloc[-3]['signal'])}" + "\n" + "\n" + "Overall Performance:" + "\n" + f"Portfolio Value: {np.round(temp_res.iloc[-1]['Portfolio Value'], 2)}" + "\n" + "\n" + text

                if running_for_the_first_time == True:
                    pass
                else:
                    SendMail(subject_text_add, text_restricted,text_nasdaq, text_tlt, text, printdf_nasdaq, printdf_tlt, pd.concat([assetsb[-1][["Ticker", "Accelerating Momentum"]].set_index("Ticker"), pd.DataFrame([unit_ticker]).transpose().rename(columns={0:"Units"})], axis=1), ["Performance^NSEI.jpg", "PerformanceTLT.jpg", "PerformanceNASDAQ.jpg"])

            # #Recalibrating Nasdaq Alpha
            # if (pd.to_datetime(date.today()) in dates_all_ss_nasdaq):
            #     print(f"Recalibrating Nasdaq at {datetime.now()}")
            #     res_test2 = select_all_strategies(24,dates_ss_nasdaq, temp_og_nasdaq, ticker_nasdaq,save=True)
            #     res_test4 = select_all_strategies(48, dates_ss_nasdaq, temp_og_nasdaq, ticker_nasdaq, save=True)
            #     # res_test8 = select_all_strategies(96, dates_ss, temp_og, ticker, save=True)
            #     ss_test_imp, res_test_imp = select_strategies_from_corr_filter(res_test2,res_test4,0, dates_ss_nasdaq, temp_og_nasdaq, number_of_optimization_periods_nasdaq,10, ticker_nasdaq, save=True)
            #
            #     res_test = []
            #     ss_test = []
            #     dates = []
            #     for date_i in range(len(dates_ss_nasdaq) - (int(24 / 3) + 1)):
            #         if (3 * date_i) % recalib_months_nasdaq == 0:
            #             dates.append(dates_ss_nasdaq[date_i + int(24 / 3)])
            #             ss_test.append(ss_test_imp[date_i])
            #             res_test.append(res_test_imp[date_i])
            #
            #     print(f"Recalibrating Weights: {datetime.now()}")
            #     inputs = []
            #     for date_i in range(len(dates)-1):
            #         inputs.append([date_i, dates, temp_og_nasdaq, ss_test, res_test, num_strategies_nasdaq, metric_nasdaq, recalib_months_nasdaq,dates_ss_nasdaq])
            #     try:
            #         pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
            #         weights_all = pool.map(optimize_weights_live, inputs)
            #     finally: # To make sure processes are closed in the end, even if errors happen
            #         pool.close()
            #         pool.join()
            #
            #     weights_update = [None]*(len(dates)-1)
            #     for date_i in range(len(dates)-1):
            #         weights_update[weights_all[date_i][0]] = weights_all[date_i][1]
            #
            #     with open(f'{ticker_nasdaq}/weights/Results_Ticker{ticker_nasdaq}_LP{number_of_optimization_periods_nasdaq}_Recal{recalib_months_nasdaq}_NS{num_strategies_nasdaq}_M{metric_nasdaq}.pkl','rb') as file:
            #         weights = pickle.load(file)
            #
            #     weights.append(weights_update[-1])
            #
            #     with open(f'{ticker_nasdaq}/weights/Results_Ticker{ticker_nasdaq}_LP{number_of_optimization_periods_nasdaq}_Recal{recalib_months_nasdaq}_NS{num_strategies_nasdaq}_M{metric_nasdaq}.pkl', 'wb') as file:
            #         pickle.dump(weights, file)
            #
            #     print(f"Recalibration for Nasdaq Over at: {datetime.now()}")
            #
            # # Recalibrating TLT Alpha
            # if (pd.to_datetime(date.today()) in dates_all_ss_tlt):
            #     print(f"Recalibrating TLT at {datetime.now()}")
            #     res_test2 = select_all_strategies(24, dates_ss_tlt, temp_og_tlt, ticker_tlt, save=True)
            #     res_test4 = select_all_strategies(48, dates_ss_tlt, temp_og_tlt, ticker_tlt, save=True)
            #     # res_test8 = select_all_strategies(96, dates_ss, temp_og, ticker, save=True)
            #     ss_test_imp, res_test_imp = select_strategies_from_corr_filter(res_test2, res_test4, 0, dates_ss_tlt,
            #                                                                    temp_og_tlt,
            #                                                                    number_of_optimization_periods_tlt,
            #                                                                    10, ticker_tlt, save=True)
            #
            #     res_test = []
            #     ss_test = []
            #     dates = []
            #     for date_i in range(len(dates_ss_tlt) - (int(24 / 3) + 1)):
            #         if (3 * date_i) % recalib_months_tlt == 0:
            #             dates.append(dates_ss_tlt[date_i + int(24 / 3)])
            #             ss_test.append(ss_test_imp[date_i])
            #             res_test.append(res_test_imp[date_i])
            #
            #     print(f"Recalibrating Weights: {datetime.now()}")
            #     inputs = []
            #     for date_i in range(len(dates) - 1):
            #         inputs.append(
            #             [date_i, dates, temp_og_tlt, ss_test, res_test, num_strategies_tlt, metric_tlt,
            #              recalib_months_tlt, dates_ss_tlt])
            #     try:
            #         pool = multiprocessing.Pool(processes=7, maxtasksperchild=1)
            #         weights_all = pool.map(optimize_weights_live, inputs)
            #     finally:  # To make sure processes are closed in the end, even if errors happen
            #         pool.close()
            #         pool.join()
            #
            #     weights_update = [None] * (len(dates) - 1)
            #     for date_i in range(len(dates) - 1):
            #         weights_update[weights_all[date_i][0]] = weights_all[date_i][1]
            #
            #     with open(
            #             f'{ticker_tlt}/weights/Results_Ticker{ticker_tlt}_LP{number_of_optimization_periods_tlt}_Recal{recalib_months_tlt}_NS{num_strategies_tlt}_M{metric_tlt}.pkl',
            #             'rb') as file:
            #         weights = pickle.load(file)
            #
            #     weights.append(weights_update[-1])
            #
            #     with open(
            #             f'{ticker_tlt}/weights/Results_Ticker{ticker_tlt}_LP{number_of_optimization_periods_tlt}_Recal{recalib_months_tlt}_NS{num_strategies_tlt}_M{metric_tlt}.pkl',
            #             'wb') as file:
            #         pickle.dump(weights, file)
            #
            #     print(f"Recalibration for TLT Over at: {datetime.now()}")
            #
            # #Recalibrating AM Constituents
            # if (pd.to_datetime(date.today()) in dates_recalibrating):
            #     print(f"Start Recalibrating constituents of NDX at {datetime.now()}")
            #     index = ".NDX"
            #     constituents = get_constituents(index)
            #
            #     with open(f'NDX_Constituents.pkl', 'wb') as file:
            #         pickle.dump(constituents, file)
            #
            #     print(f"End Recalibrating constituents of NDX at {datetime.now()}")
            #
            # #Recalibrating AM
            # if (pd.to_datetime(date.today()) in dates_recalibrating):
            #     print(f"Start Recalibrating AM of NDX at {datetime.now()}")
            #     assets.append(get_weights_stocks_live(constituents, top_nassets, recalibrating_months, training_period,dates_recalibrating, data_inp, save=False)[0])
            #     with open(f'NASDAQ_RecalibPeriod_{int(1)}.pkl', 'wb') as file:
            #         pickle.dump(assets, file)
            #     print(f"End Recalibrating AM of NDX at {datetime.now()}")

        if running_for_the_first_time:
            pass
        else:
            if datetime.now() < datetime.now().replace(hour=run_hour).replace(minute=run_minute).replace(second=30):
                continue

        running_for_the_first_time = False

        print(f"Sleeping: {datetime.now()}")

        time_now = datetime.now()
        next_run = datetime.now()
        try:
            if (datetime.now().hour < run_hour) & (datetime.now().minute < run_minute):
                next_run = next_run.replace(day=next_run.day).replace(hour=run_hour).replace(minute=run_minute).replace(
                    second=00)
            else:
                next_run = next_run.replace(day=next_run.day + 1).replace(hour=run_hour).replace(
                    minute=run_minute).replace(second=00)
        except:
            if next_run.month == 12:
                next_run = next_run.replace(year=next_run.year + 1).replace(month=1).replace(day=1).replace(
                    hour=run_hour).replace(minute=run_minute).replace(second=00)
            else:
                next_run = next_run.replace(day=1).replace(month=next_run.month + 1).replace(hour=run_hour).replace(
                    minute=run_minute).replace(second=00)

        print(f"Supposed to wake up at: {datetime.now() + timedelta(seconds=(next_run - time_now).seconds - 150)}")
        time.sleep((next_run - time_now).seconds-150)
        print(f"Woken Up: {datetime.now()}")





